# OData demo

Resource: https://www.odata.org/odata-services/

In [11]:
import requests
from requests.auth import HTTPBasicAuth

import os
from dotenv import load_dotenv

load_dotenv()


# Your OData service URL
url = 'https://services.odata.org/TripPinRESTierService/$metadata'


# Make a GET request with basic authentication
metadata_response = requests.get(url)

# Check if the request was successful
if metadata_response.status_code == 200:
    # Process the response data (assuming it's JSON)
    print(f"Succeeded to retrieve data: {metadata_response.status_code}")
else:
    print(f"Failed to retrieve data: {metadata_response.status_code}")


Succeeded to retrieve data: 200


In [27]:
# check there is content in the response
print(metadata_response.content)

b'<?xml version="1.0" encoding="utf-8"?><edmx:Edmx Version="4.0" xmlns:edmx="http://docs.oasis-open.org/odata/ns/edmx"><edmx:DataServices><Schema Namespace="Trippin" xmlns="http://docs.oasis-open.org/odata/ns/edm"><EntityType Name="Person"><Key><PropertyRef Name="UserName" /></Key><Property Name="UserName" Type="Edm.String" Nullable="false" /><Property Name="FirstName" Type="Edm.String" Nullable="false" /><Property Name="LastName" Type="Edm.String" MaxLength="26" /><Property Name="MiddleName" Type="Edm.String" /><Property Name="Gender" Type="Trippin.PersonGender" Nullable="false" /><Property Name="Age" Type="Edm.Int64" /><Property Name="Emails" Type="Collection(Edm.String)" /><Property Name="AddressInfo" Type="Collection(Trippin.Location)" /><Property Name="HomeAddress" Type="Trippin.Location" /><Property Name="FavoriteFeature" Type="Trippin.Feature" Nullable="false" /><Property Name="Features" Type="Collection(Trippin.Feature)" Nullable="false" /><NavigationProperty Name="Friends" Typ

## Understanding OData metadata structure

The metadata of an OData service  is in **XML format** describing the data model (EDM - Entity Data Model) of the service (entities, relationships, functions, etc.). Since this metadata is not JSON, but XML, we would need to parse the XML content to read the data.

The *edm* namespace is specifically used within OData *metadata* documents (`$metadata`) to define the structure and schema of the data exposed by the OData service.

-------------------

 The metadata document, returned in XML format, describes the **data model** of the OData service, including its **entity sets**, **entity types**, **complex types**, **associations**, **functions**, and more. 


- **EntityContainer** 

The EntityContainer is the entry point for the data model and encapsulates the entity sets, function imports, and singletons available in the service. It represents the set of entities and operations exposed by the service.

- **EntitySet**

An EntitySet represents a collection of entities of the same type. For example, a "Products" entity set would represent all product entities available in the service.

- **EntityType**

An EntityType defines the structure of an entity, including its properties and keys. It's similar to defining a class in object-oriented programming, where the properties represent the data fields of the entity.

*Key*: uniquely identifies the entity within an entity set (similar to **primary keys**) -> Can be the PropertyRef

*PropertyRef*: points to a property (or a set of properties) within an entity type that serves as the unique identifier for entities of that type

*Property*: represents the attribute of an entity (similar to how **columns** define the schema of a table in relational DBs)


<EntityType Name="Product">
    <Key>
        <PropertyRef Name="ProductID" />
    </Key>
    <Property Name="ProductID" Type="Edm.Int32" Nullable="false"/>

</EntityType>



- **ComplexType**

A ComplexType is used to define a structured type that doesn't have a key and is used to group properties. It can be used within entity types or other complex types.

- **Association** Properties 

Associations define the *relationships* between entity types.

- **Navigation** Properties

Navigation properties are used within entity types to navigate to *associations*. Access navigations via `$expand`, e.g., `GET /Authors(1)?$expand=WrittenBooks`

- **Function** and **Action**

Functions and actions represent operations that can be performed on the data. *Functions* are operations that *don't have side effects*, whereas *actions might modify data*.


--------------------
The namespace `http://docs.oasis-open.org/odata/ns/edm` for the Entity Data Model (EDM) is **standard across all OData services for elements and attributes that are part of the OData version 4 specification defined by the OASIS consortium. This means that whenever you see this namespace in an OData service's $metadata document, it signifies elements that are consistent with the OData version 4 specifications, such as entity types, entity sets, complex types, functions, actions, and more.

However, **custom namespaces** can be defined by OData services to extend the EDM with custom elements and attributes. These custom namespaces are typically used to define service-specific elements and attributes.

In [12]:
import xml.etree.ElementTree as ET

xml_metadata = ""

def save_XML_data(tag, attribute, XML_data):

    short_tag = tag.split("}")[1]

    if short_tag == "Edmx" or short_tag == "DataServices" or short_tag == "Schema":
        return XML_data

    if short_tag == 'EntityType':
        XML_data += "\n"
    XML_data += f"{short_tag} {attribute}\n"

    return XML_data


# Check if the request was successful
if metadata_response.status_code == 200:
    # Parse the XML response
    root = ET.fromstring(metadata_response.content)
             
    # Now you can work with the XML data
    # For example, print out the XML tree structure
    for entity in root.iter():        

        xml_metadata = save_XML_data(entity.tag, entity.attrib, xml_metadata)
        
    print("Succeeded to retrieve data\n")
else:
    print(f"Failed to retrieve data: {metadata_response.status_code}")

print(xml_metadata)


Succeeded to retrieve data


EntityType {'Name': 'Person'}
Key {}
PropertyRef {'Name': 'UserName'}
Property {'Name': 'UserName', 'Type': 'Edm.String', 'Nullable': 'false'}
Property {'Name': 'FirstName', 'Type': 'Edm.String', 'Nullable': 'false'}
Property {'Name': 'LastName', 'Type': 'Edm.String', 'MaxLength': '26'}
Property {'Name': 'MiddleName', 'Type': 'Edm.String'}
Property {'Name': 'Gender', 'Type': 'Trippin.PersonGender', 'Nullable': 'false'}
Property {'Name': 'Age', 'Type': 'Edm.Int64'}
Property {'Name': 'Emails', 'Type': 'Collection(Edm.String)'}
Property {'Name': 'AddressInfo', 'Type': 'Collection(Trippin.Location)'}
Property {'Name': 'HomeAddress', 'Type': 'Trippin.Location'}
Property {'Name': 'FavoriteFeature', 'Type': 'Trippin.Feature', 'Nullable': 'false'}
Property {'Name': 'Features', 'Type': 'Collection(Trippin.Feature)', 'Nullable': 'false'}
NavigationProperty {'Name': 'Friends', 'Type': 'Collection(Trippin.Person)'}
NavigationProperty {'Name': 'BestFriend', 'Type': 'Tri

https://docs.python.org/3/library/xml.etree.elementtree.html#xml.etree.ElementTree.ElementTree.write

(formatted the indentation of XML in Notepad++)

In [10]:
#optional; Debugging purposes

tree = ET.ElementTree(root)

tree.write("output.xml")

### ~~~Save Metadata - DRAFT~~~

https://www.mssqltips.com/sqlservertip/2899/importing-and-processing-data-from-xml-files-into-sql-server-tables/

In SQL Server Management Studio created new table

```
CREATE TABLE XML_metadata
(
	Id INT IDENTITY PRIMARY KEY, 
	XMLData XML, 
	LoadedDateTime DATETIME
)
```



# Formulate query

In [13]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key),
)



Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'chat_completion': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x000002D9A0218A40>}, chat_services={'chat_completion': <function Kernel.add_chat_service.<locals>.<lambda> at 0x000002D9A02189A0>}, text_embedding_generation_services={}, default_text_completion_service='chat_completion', default_chat_service='chat_completion', default_text_embedding_generation_service=None, retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

### AOAI model analyze schema and user request

Get from Schema the following information: TableName, ColumnName and ConstraintType

In [35]:
sk_prompt_analyze_metadata = """
We have a database that is accessible via OData service ({{$odata_service_api}})). We have access to the metadata. Based on the {{$metadata}} and user request {{$user_request}}, we need to generate an API request to send back to te database in order to retrieve the required information to answer the user request. 
Try to make the API request as specific as possible (retrieve only the relevant fields, for example NOT the IDs), and include the parameters that are required to retrieve the information. Only make use of the parameters that are available in the metadata.
Include the base URL. Give only the query that can be directly run against the database. No other explanations. 
"""

chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt_analyze_metadata,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0,
    top_p=0.5,
)

context = kernel.create_new_context()
context["odata_service_api"] = "https://services.odata.org/TripPinRESTierService"
context["metadata"] = xml_metadata
context["user_request"] = "what trips did russel whyte do?"

bot_answer = await chat_function.invoke(context=context)

print(bot_answer)

https://services.odata.org/TripPinRESTierService/People('russellwhyte')/Trips?$select=Name,Description,StartsAt,EndsAt


## Get answer from DB

In [36]:
# Your OData service URL
url = bot_answer


# Make a GET request with basic authentication
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Process the response data (assuming it's JSON)
    print(f"Succeeded to retrieve data: {response.status_code}")

    
    print(response.content)
else:
    print(f"Failed to retrieve data: {response.status_code}")



Succeeded to retrieve data: 200
b'{"@odata.context":"https://services.odata.org/TripPinRESTierService/(S(c42thhn11snkh4vvvspvyweo))/$metadata#Collection(Trippin.Trip)","value":[{"Name":"Trip in US","Description":"Trip from San Francisco to New York City","StartsAt":"2014-01-01T00:00:00Z","EndsAt":"2014-01-04T00:00:00Z"},{"Name":"Trip in Beijing","Description":"Trip from Shanghai to Beijing","StartsAt":"2014-02-01T00:00:00Z","EndsAt":"2014-02-04T00:00:00Z"},{"Name":"Honeymoon","Description":"Happy honeymoon trip","StartsAt":"2014-02-01T00:00:00Z","EndsAt":"2014-02-04T00:00:00Z"}]}'


To make the response cleaner and shorter:

In [37]:
import json

response_dict = json.loads(response.text)

response_dict.pop('@odata.context')

string_answer = json.dumps(response_dict)

string_answer = string_answer.replace("\"", '')

print(string_answer)

{value: [{Name: Trip in US, Description: Trip from San Francisco to New York City, StartsAt: 2014-01-01T00:00:00Z, EndsAt: 2014-01-04T00:00:00Z}, {Name: Trip in Beijing, Description: Trip from Shanghai to Beijing, StartsAt: 2014-02-01T00:00:00Z, EndsAt: 2014-02-04T00:00:00Z}, {Name: Honeymoon, Description: Happy honeymoon trip, StartsAt: 2014-02-01T00:00:00Z, EndsAt: 2014-02-04T00:00:00Z}]}


## Generate answer back to the user based on retrieved data

In [38]:
sk_prompt_reply_user = """
Use the given information (retrieved_data) to answer the user's request. User request: {{$user_request}}. Retrieved_data: {{$retrieved_data}}.
"""

reply_function = kernel.create_semantic_function(
    prompt_template=sk_prompt_reply_user,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.5,
    top_p=0.5,
)

context["retrieved_data"] = string_answer

bot_answer = await reply_function.invoke(context=context)

print(bot_answer)

Russel Whyte did the following trips:

1. A trip in the US from San Francisco to New York City. This trip started on January 1, 2014, and ended on January 4, 2014.

2. A trip in Beijing from Shanghai to Beijing. This trip started on February 1, 2014, and ended on February 4, 2014.

3. A honeymoon trip. The details of the location are not specified, but it started on February 1, 2014, and ended on February 4, 2014.
